In [1]:
import os
import re
import copy
import numpy
from IPython.display import Image, display
from rmgpy.data.thermo import ThermoLibrary
from rmgpy.data.rmg import RMGDatabase
from rmgpy import settings
from rmgpy.species import Species
from rmgpy.molecule import Molecule
from rmgpy.cantherm.output import prettify

In [80]:
def addThermoData(thermoData1, thermoData2):
    """
    Add the thermodynamic data `thermoData2` to the data `thermoData1`,
    and return `thermoData1`.
    """
    if len(thermoData1.Tdata.value_si) != len(thermoData2.Tdata.value_si) or any([T1 != T2 for T1, T2 in zip(thermoData1.Tdata.value_si, thermoData2.Tdata.value_si)]):
        raise Exception('Cannot add these ThermoData objects due to their having different temperature points.')

    for i in range(thermoData1.Tdata.value_si.shape[0]):
        thermoData1.Cpdata.value_si[i] += thermoData2.Cpdata.value_si[i]
    thermoData1.H298.value_si += thermoData2.H298.value_si
    thermoData1.S298.value_si += thermoData2.S298.value_si

    #if thermoData1.comment:
    #    thermoData1.comment += ' + {0}'.format(thermoData2.comment)
    #else:
    #    thermoData1.comment = 'Thermo group additivity estimation: ' + thermoData2.comment

    return thermoData1

def removeThermoData(thermoData1, thermoData2):
    """
    Remove the thermodynamic data `thermoData2` from the data `thermoData1`,
    and return `thermoData1`.
    """
    if len(thermoData1.Tdata.value_si) != len(thermoData2.Tdata.value_si) or any([T1 != T2 for T1, T2 in zip(thermoData1.Tdata.value_si, thermoData2.Tdata.value_si)]):
        raise Exception('Cannot add these ThermoData objects due to their having different temperature points.')

    for i in range(thermoData1.Tdata.value_si.shape[0]):
        thermoData1.Cpdata.value_si[i] -= thermoData2.Cpdata.value_si[i]
    thermoData1.H298.value_si -= thermoData2.H298.value_si
    thermoData1.S298.value_si -= thermoData2.S298.value_si

    #if thermoData1.comment:
    #    thermoData1.comment += ' + {0}'.format(thermoData2.comment)
    #else:
    #    thermoData1.comment = 'Thermo group additivity estimation: ' + thermoData2.comment

    return thermoData1

def averageThermoData(thermoDataset=[]):
    """
    Average a list of thermoData values together.
    Sets uncertainty values to be the approximately the 95% confidence interval, equivalent to
    2 standard deviations calculated using the sample standard variance:
    
    Uncertainty = 2s
    s = sqrt( sum(abs(x - x.mean())^2) / N - 1) where N is the number of values averaged
    
    Note that uncertainties are only computed when number of values is greater than 1.
    """
    
    numValues = len(thermoDataset)
        
    if numValues == 0:
        raise Exception('No thermo data values were inputted to be averaged.')
    else:
        print 'Averaging thermo data over {0} value(s).'.format(numValues)
        
        if numValues == 1:
            return copy.deepcopy(thermoDataset[0])
        
        else:
            averagedThermoData = copy.deepcopy(thermoDataset[0])
            for thermoData in thermoDataset[1:]:
                averagedThermoData = addThermoData(averagedThermoData, thermoData)


            for i in range(averagedThermoData.Tdata.value_si.shape[0]):
                averagedThermoData.Cpdata.value_si[i] /= numValues
                #print averagedThermoData.Cpdata.uncertainty
                cpData = [thermoData.Cpdata.value_si[i] for thermoData in thermoDataset]
                averagedThermoData.Cpdata.uncertainty[i] = 2*numpy.std(cpData, ddof=1)

            HData = [thermoData.H298.value_si for themoData in thermoDataset]
            averagedThermoData.H298.value_si /= numValues
            #print averagedThermoData.H298.getUncertainty()
            #print averagedThermoData.H298.uncertainty
            #print averagedThermoData.H298.uncertainty_si
            averagedThermoData.H298.uncertainty_si = 2*numpy.std(HData, ddof=1)

            SData = [thermoData.S298.value_si for themoData in thermoDataset]
            averagedThermoData.S298.value_si /= numValues
            averagedThermoData.S298.uncertainty_si = 2*numpy.std(SData, ddof=1)
            return averagedThermoData
        
def extractPolycyclicGroups(molecule):
    """
    Extract polycyclic functional groups from a real molecule
    """
    struct = molecule.copy(deep=True)
    
    # Saturate the structure if it is a radical
    if struct.isRadical():
        struct.saturate()
    struct.deleteHydrogens()
    print struct.toAdjacencyList()
    polyRings = struct.getPolycyclicRings()
    groups = [convertCycleToGroup(ring) for ring in polyRings]
    
    return groups
                
def convertCycleToGroup(cycle):
    """
    This function converts a list of atoms in a cycle to a functional Group object
    """
    from rmgpy.molecule.group import GroupAtom, GroupBond, Group
    
    # Create GroupAtom object for each atom in the cycle
    groupAtoms = {}
    bonds = []
    for atom in cycle:
        groupAtoms[atom] = GroupAtom(atomType=[atom.atomType])
        
        # Get all the bonds between atoms in the cycle but not outside of the cycle
        for bondedAtom, bond in atom.edges.iteritems():
            if bondedAtom in cycle:
                bonds.append(bond)
                
                
    # Create GroupBond for each bond between atoms in the cycle, but not outside of the cycle
    for atom in cycle:
        for bondedAtom, bond in atom.edges.iteritems():
            if bondedAtom in cycle:
                # create a group bond mimicking the bond in the molecule
                GroupBond(groupAtoms[atom],groupAtoms[bondedAtom])
            else:
                pass
        print groupAtoms[atom].edges
        
    return Group(atoms=groupAtoms.values())

In [82]:
m = Molecule(SMILES='C1CC2CCC1C2')
groups = extractPolycyclicGroups(m)
print groups[0].toAdjacencyList()


1 C u0 p0 c0 {2,S} {6,S}
2 C u0 p0 c0 {1,S} {3,S}
3 C u0 p0 c0 {2,S} {4,S} {7,S}
4 C u0 p0 c0 {3,S} {5,S}
5 C u0 p0 c0 {4,S} {6,S}
6 C u0 p0 c0 {1,S} {5,S} {7,S}
7 C u0 p0 c0 {3,S} {6,S}

{}
{}
{}
{}
{}
{}
{}
1 Cs ux
2 Cs ux
3 Cs ux
4 Cs ux
5 Cs ux
6 Cs ux
7 Cs ux



In [3]:
# Fill in the list of thermo libraries to be used for fitting polycyclic thermo groups
thermoLibraries = ['C10H11']
database = RMGDatabase()
database.load(settings['database.directory'], thermoLibraries = thermoLibraries, kineticsFamilies='none', kineticsDepositories='none', reactionLibraries=[])

thermoDatabase = database.thermo

In [4]:
fittingDictionary={}
for libraryName in thermoLibraries:
    thermoLibrary = database.thermo.libraries[libraryName]

In [19]:
for label, entry in thermoLibrary.entries.iteritems():
    molecule = entry.item
    libraryThermoData = entry.data
    if molecule.getAllPolycyclicVertices():
        species = Species(molecule=[molecule])
        species.generateResonanceIsomers() 
        estimatedThermo = thermoDatabase.getThermoDataFromGroups(species)
        
        tokens = estimatedThermo.comment.split()
        polycyclicGroups = []
        for token in tokens:
            if 'polycyclic' in token:
                splitTokens = re.split("\(|\)",token)
                assert len(splitTokens) == 3
                groupLabel = splitTokens[1]
                polycyclicGroups.append(thermoDatabase.groups['polycyclic'].entries[groupLabel])

        if len(polycyclicGroups) == 0:
            raise Exception('Species {0} detected as polycyclic but estimated thermo contained no polycyclic groups: \
                    you need to create a new polycyclic group'.format(label))
            
        elif len(polycyclicGroups) == 1:
            polycyclicGroup = polycyclicGroups[0]
            print 'Species {0} has a single polycyclic group match in thermo estimate: {1}'.format(label, polycyclicGroup.label)
            # Draw the molecule in ipython notebook
            #display(molecule)
            
            withoutPolycyclicGroupThermo = removeThermoData(estimatedThermo, polycyclicGroup.data)
            newPolycyclicGroupThermo = removeThermoData(libraryThermoData, withoutPolycyclicGroupThermo)
            # At this point the estimatedThermo and librarythermoData are permanently modified and should not be reused
            
            
            # Check to make sure that the polycyclic group is not generic
            if polycyclicGroup.label == 'PolycyclicRing':
                group = extractPolycyclicGroup(molecule)

                
            
            
            
            
            
            # Add the new group value to the fitting dictionary
            if polycyclicGroup not in fittingDictionary:
                # Add a tuple containing fitted group data, the original library entry, and thermo library
                fittingDictionary[polycyclicGroup]=[(newPolycyclicGroupThermo, entry, thermoLibrary)]
            else:
                fittingDictionary[polycyclicGroup].append((newPolycyclicGroupThermo, entry, thermoLibrary))
                
        elif len(polycyclicGroups) > 1:
            print 'Species {0} has matched multiple polycyclic groups. \
                    This cannot be fitted with a single molecule\'s thermo data.'.format(label)
            raise Exception


Species pdt7 has a single polycyclic group match in thermo estimate: PolycyclicRing


NameError: name 'extractPolycyclicGroup' is not defined

In [11]:
for polycyclicGroup, fittingGroups in fittingDictionary.iteritems():
    print 'Original thermo data for polycyclic group: {0}'.format(polycyclicGroup.label)
    print prettify(repr(polycyclicGroup.data))
    
    thermoDataset = [fitTuple[0] for fitTuple in fittingGroups]
    labels = [fitTuple[1].label for fitTuple in fittingGroups]
    libraryLabels = [fitTuple[2].name for fitTuple in fittingGroups]
    # Average the new group values to fit the original polycyclic group
    fittedGroupData = averageThermoData([fitTuple[0] for fitTuple in fittingGroups])
    #print fittedGroupData
    #print fittingGroups
    polycyclicGroup.data = fittedGroupData
    polycyclicGroup.shortDesc = "Fitted from thermo library values"
    
    comment = ''
    for i in range(len(labels)):
        comment += "Fitted from {0} species from {1} library.\n".format(labels[i],libraryLabels[i])
    polycyclicGroup.longDesc = comment.strip()
    
    print 'Fitted thermo data for polycyclic group: {0}'.format(polycyclicGroup.label)
    print prettify(repr(polycyclicGroup.data))
    print polycyclicGroup.longDesc
    print '===================================================================='
    # At this point, save and overwrite the entire polycyclic thermo library

Original thermo data for polycyclic group: PolycyclicRing
ThermoData(
    Tdata = ([300, 400, 500, 600, 800, 1000, 1500], 'K'),
    Cpdata = ([0, 0, 0, 0, 0, 0, 0], 'cal/(mol*K)'),
    H298 = (0, 'kcal/mol'),
    S298 = (0, 'cal/(mol*K)'),
    comment = 'polycyclic(PolycyclicRing)',
)
Averaging thermo data over 66 value(s).
Fitted thermo data for polycyclic group: PolycyclicRing
ThermoData(
    Tdata = ([300, 400, 500, 600, 800, 1000, 1500], 'K'),
    Cpdata = ([-92.7864, -117.575, -137.134, -151.643, -173.244, -187.852, -209.689], 'cal/(mol*K)', '+|-', [60.8345, 66.4408, 69.4059, 70.3875, 75.439, 81.5102, 91.9894]),
    H298 = (-105.053, 'kcal/mol'),
    S298 = (-4.66189, 'cal/(mol*K)', '+|-', 1.369e-14),
)
Fitted from pdt7 species from C10H11 library.
Fitted from pdt8 species from C10H11 library.
Fitted from pdt10bis species from C10H11 library.
Fitted from pdt11 species from C10H11 library.
Fitted from pdt17 species from C10H11 library.
Fitted from pdt19 species from C10H11 library.